# CeneoScarper

## Lista kroków "manualnych"

1. Otworzyć Ceneo
2. Wyszukać produkt
3. Otworzyć okno produktu
4. Wybrać zakładkę Opinie i Recencje
5. Skopiować opinie
6. Wkleić opinie

## Lista kroków scrapera

1. Wysłanie żądania dostępu do zasobu
2. Wydobycie z kodu HTML fragmentów odpowiadających opiniom
3. Wydobycie z kodu opinii jej składowych do słownika dla każdej opinii na stronie
4. Dodanie słownika reprezentującego pojedynczą opinię do listy

## Struktura opinii w serwisie Ceneo.pl

|składowa|nazwa|selektor|
|--------|-----|--------|
|identyfikator opinii|opinion_id|["data-entry-id"]|
|autor|author|span.user-post__author-name|
|rekomendacja|recommendation|span.user-post__author-recomendation > em|
|liczba gwiazdek|stars|span.user-post__score-count|
|treść opinii|content|div.user-post__text|
|lista zalet|pros|div.review-feature__title--positives ~ div.review-feature__item|
|lista wad|cons|div.review-feature__title--negatives ~ div.review-feature__item|
|data wystawienia opinii|post_date|span.user-post__published > time:nth-child(1)["datetime"]|
|data zakupu produktu|purchase_date|span.user-post__published > time:nth-child(2)["datetime"]|
|ile osób uznało opinię za przydatną|useful|button.vote-yes > span|
|ile osób uznało opinię za nieprzydatną|useless|button.vote-no > span|

49729622
61161229
36068712
37089065
92504536

1. Import bibliotek

In [44]:
import os
import json
import requests
from bs4 import BeautifulSoup

2. Wysłanie żądania dostępu do zasobu

In [45]:
product_code = "92504536"
url = f"https://www.ceneo.pl/{product_code}#tab=reviews"
response = requests.get(url)

3. Wydobycie z kodu HTML fragmentów odpowiadających opiniom

In [46]:
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review")

4. Wydobycie z kodu opinii jej składowych dla każdej opinii na stronie

In [47]:
all_opinions = []
for opinion in opinions:
    single_opinion = {
        'opinion_id': opinion["data-entry-id"].strip(),
        'author': opinion.select_one("span.user-post__author-name").text.strip(),
        'recommendation': opinion.select_one("span.user-post__author-recomendation > em").text.strip(),
        'stars': opinion.select_one("span.user-post__score-count").text.strip(),
        'content': opinion.select_one("div.user-post__text").text.strip(),
        'pros': [p.text for p in opinion.select("div.review-feature__title--positives ~ div.review-feature__item")],
        'cons': [c.text for c in opinion.select("div.review-feature__title--negatives ~ div.review-feature__item")],
        'post_date': opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"].strip(),
        # 'purchase_date': opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"].strip(),
        'useful': opinion.select_one("button.vote-yes > span").text.strip(),
        'useless': opinion.select_one("button.vote-no > span").text.strip(),
    }
all_opinions.append(single_opinion)

5. Zapis opinii do pliku JSON

In [48]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
file = open(f"opinions/{product_code}.json", "w", encoding="UTF-8")
json.dump(all_opinions, file, indent=4, ensure_ascii=False)
file.close()